# 🎬 VEDA Video Engine - Lightweight Version
### Uses AnimateLCM - Works on Free Colab!

---
1. **Runtime → Change runtime type → T4 GPU**
2. Run all cells in order

In [ ]:
#@title 1️⃣ Install & Setup (3 min)
!pip install -q diffusers transformers accelerate
!pip install -q imageio imageio-ffmpeg

import torch
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'None'}")
print("✅ Ready!")

In [ ]:
#@title 2️⃣ Load AnimateLCM Model (2-3 min)
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter

print("Loading lightweight video model...")

# Load motion adapter
adapter = MotionAdapter.from_pretrained(
    "wangfuyun/AnimateLCM",
    torch_dtype=torch.float16
)

# Load pipeline with small SD model
pipe = AnimateDiffPipeline.from_pretrained(
    "emilianJR/epiCRealism",
    motion_adapter=adapter,
    torch_dtype=torch.float16,
)

pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config)
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

print("\n✅ Model loaded! (Uses ~4GB VRAM)")

In [ ]:
#@title 3️⃣ Generate Video 🎬
import torch
import gc

#@markdown ### Your Prompt
prompt = "beautiful sunset over ocean, golden light, waves, cinematic" #@param {type:"string"}

#@markdown ### Settings
num_frames = 16 #@param {type:"slider", min:8, max:24, step:4}
steps = 6 #@param {type:"slider", min:4, max:8, step:1}
seed = 42 #@param {type:"integer"}

gc.collect()
torch.cuda.empty_cache()

print(f"🎬 Generating: {prompt[:40]}...")
print(f"⏳ Takes ~30 seconds with LCM...\n")

output = pipe(
    prompt=prompt,
    negative_prompt="blurry, ugly, low quality",
    num_frames=num_frames,
    num_inference_steps=steps,
    guidance_scale=2.0,
    generator=torch.Generator("cpu").manual_seed(seed),
)

frames = output.frames[0]
print(f"\n✅ Generated {len(frames)} frames!")

In [ ]:
#@title 4️⃣ Save & Preview
import imageio
from IPython.display import Video, display
import numpy as np
import os

os.makedirs("/content/outputs", exist_ok=True)
output_path = "/content/outputs/veda_video.mp4"

frames_np = [np.array(f) for f in frames]
imageio.mimsave(output_path, frames_np, fps=8)

print(f"✅ Saved: {output_path}")
display(Video(output_path, embed=True, width=512))

In [ ]:
#@title 5️⃣ Download
from google.colab import files
files.download(output_path)
print("📥 Downloading...")

## 🎨 Prompt Ideas
- `beautiful woman smiling, portrait, soft lighting`
- `ocean waves on beach, sunset, peaceful`
- `city street at night, neon lights, rain`
- `flower blooming, timelapse, nature`